# BART Sequence Programming, Simulation, Reconstruction

Based off of previous tutorials from other contributors:  

1. [MRITogether 2023 Tutorial](https://github.com/mrirecon/bart-workshop/tree/master/mri_together_2023), [Jon Tamir](mailto:jtamir@utexas.edu), Chandra Family Department of Electrical and Computer Engineering, The University of Texas at Austin
2. [MRITogether 2021 Tutorial](https://github.com/mrirecon/bart-workshop/tree/master/mri_together_2021), [Martin Uecker](mailto:uecker@tugraz.at)$^{\dagger}$, Nick Scholand, Moritz Blumenthal, Xiaoqing Wang, $^{\dagger}$ Graz University of Technology
3. [BART Webinar 1 Tutorial](https://github.com/mrirecon/bart-webinars/tree/master/webinar1), [Max Litster](mailto:maxlitster@berkeley.edu), UC Berkeley
4. [Open MR Days 2025 Workshop](https://github.com/mrirecon/bart-workshop/tree/master/dach_ismrm2025), [Martin Uecker](mailto:uecker@tugraz.at), [Daniel Mackner](mailto:daniel.mackner@tugraz.at), [Vitali Telezki](mailto:vitali.telezki@med.uni-goettingen.de)


The purpose of this notebook is to introduce the [BART toolbox](https://mrirecon.github.io/bart), and its use as a tool for reproducible research in MRI.

We recommend checking other points of reference, primarily available in the following locations:
- [BART Workshops](https://github.com/mrirecon/bart-workshop)
- [BART Webinars](https://github.com/mrirecon/bart-webinars) (and [webinar recordings](https://www.youtube.com/@bart-toolboxdepartmental7435/playlists)\)


### Local Usage
- Install bart from its [github repository](https://github.com/mrirecon/bart)
- Set the `BART_TOOLBOX_PATH` to the BART directory and add it to the `PATH`

```bash
export BART_TOOLBOX_PATH=/path/to/bart  
export PATH=$BART_TOOLBOX_PATH:$PATH
```

Although the simplest way to call the BART CLI tools is from a terminal, there are also wrapper functions that allow the tools to be used from Matlab and Python. These are located under the `$BART_TOOLBOX_PATH/matlab` and `$BART_TOOLBOX_PATH/python` directories.

### Online Usage
We recommend running BART locally when possible. For demonstration purposes, MyBinder and Google Colaboratory can be used to access a Jupyter instance with BART with a browser. In the following we install and configure BART for both.

The following code will automatically detect which service you are using.

In [ ]:
# Check if notebook runs on colab
import sys, os

os.environ['COLAB'] = 'true' if ('google.colab' in sys.modules) else 'false'

### Google Colab
The cell will setup BART on Google Colab. For a detailed explanation, see **How to Run BART on Google Colaboratory** [here](https://github.com/mrirecon/bart-workshop/blob/master/ismrm2021/bart_on_colab/colab_gpu_tutorial.ipynb). You can skip this part if you want to run this notebook on your local machine.

This tutorial do need a GPU, you can select one by going to **Edit → Notebook settings**: Choose **GPU** from **Hardware accelerator** drop-down menu.

In [ ]:
%%bash

if $COLAB; then
# Why Google?! As of May 13 cuda installed in colab (12.5) is incompatible with the driver (12.4)
# We install cuda 12.4 and link it here but it takes time!

#apt install cuda-12-4
#
#cd /usr/local
#rm cuda
#ln -s cuda-12.4 cuda
#
echo "GPU Information:"
nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv
nvcc --version

fi

#### BART Installation

Here we install BARTs dependencies, clone its repository from github, and compile it.

Check BART setup:

In [ ]:
%%bash

# MyBinder has BART already installed via the container
if $COLAB; then

  # Install BARTs dependencies
  apt-get install -y -qq make gcc-12 g++-12 libfftw3-dev liblapacke-dev libpng-dev libopenblas-dev

  # Clone Bart
  [ -d /content/bart ] && rm -r /content/bart
  git clone -q https://github.com/mrirecon/bart/ bart
fi

In [ ]:
%%bash

if $COLAB; then

cd bart

# Configuration
cat > Makefile.local <<HERE
PARALLEL=1
CUDA=0
CUDA_BASE=/usr/local/cuda
CUDA_LIB=lib64
OPENBLAS=1
BLAS_THREADSAFE=1
CC=gcc-12
HERE

# Compile BART
make --silent && echo ok
fi

#### Setup Environment for BART

After downloading and compiling BART, the next step simplifies the handling of BARTs command line interface inside an ipyhton jupyter notebook. We add the BART directory to the PATH variable and include the python wrapper for reading *.cfl files:

In [ ]:
if os.environ['COLAB'] == 'true':
    os.environ['BART_TOOLBOX_PATH'] = "./bart"
    os.environ['PATH'] = os.environ['BART_TOOLBOX_PATH'] + ":" + os.environ['PATH']

# Make BART python wrappers available:
sys.path.append(os.environ['BART_TOOLBOX_PATH'] + "/python/")
# Sensible number of threads for multithreading in BART
os.environ['OMP_NUM_THREADS'] = "4"

In [ ]:
%%bash
echo "# BART version: "
bart version

### Setup Visualization Helper

For this tutorial we will visualize some images. Therefore, we need a helper function and some python libraries.


In [ ]:
# More python libraries
import cfl
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import Image

def show_img(data, title=None, vmin=None, vmax=None, cmap='gray', cbar_label='', mag=True, fsize=10, \
             interpolation='bilinear', cbar=False):

    # Import data
    data = np.abs(data) if mag else data

    fig = plt.figure(figsize=(fsize,fsize))
    ax1 = fig.add_subplot(111)
    im = ax1.imshow(data, interpolation=interpolation, cmap=cmap, vmin=vmin, vmax=vmax)
    if title:
      plt.title(title)
    # Style settings
    if cbar:
        divider = make_axes_locatable(ax1)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        cbar = plt.colorbar(im, cax=cax)
        cbar.set_label(cbar_label)
        cbar.ax.tick_params()

    ax1.set_yticklabels([])
    ax1.set_xticklabels([])
    ax1.xaxis.set_ticks_position('none')
    ax1.yaxis.set_ticks_position('none')
    ax1.set_axis_off()

    plt.show()


def plot_mom_from_cfl(name, end=-1, ylim=-1):
    """Plot three axes from a BART cfl"""

    if not os.path.exists(name + '.cfl'):
        FileNotFoundError(f"{name + '.cfl'} does not exist")
    if not os.path.exists(name + '.hdr'):
        FileNotFoundError(f"{name + '.hdr'} does not exist")

    data = np.array(cfl.readcfl('./' + name))
    time_points=np.linspace(0, 1, np.shape(data)[1])
    xlabel = "time / a.u."
    if 0 < end:
        time_points=np.linspace(0, end, np.shape(data)[1])
        xlabel = "time / s"

    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 3))
    for i in range(3):
        ax[i].plot(time_points, np.real(data[i,:]))
        ax[i].set_xlabel(xlabel)

    titles = ['PE', 'RO', 'SL']
    for i in range(3):
        ax[i].grid()
        if ylim > 0:
                ax[i].set_ylim(-ylim,ylim)
        ax[i].set_title(titles[i])
    

def plot_grad_from_cfl(name, end=-1):
    """Plot three axes from a BART cfl"""

    plot_mom_from_cfl(name, end, 25e-3)

## BART Fundamentals
BART provides a number of tools for MRI image reconstruction and multi-dimensional array manipulation. We can get a list of them by running `bart` with no arguments.

Note: The ! character indicates that the following cell should be executed in bash vs. the native python kernel. It is not necessary when running BART commands from your terminal.



In [ ]:
# get a list of BART commands by running bart with no arguments:
! bart

### BART Command Structure

All BART commands are structured in the following way:

> **`bart`** + **`command`** + **`options`** + **`input / output`**

Each BART command consists of a number of optional arguments, followed by input and output files. To get some information about a command, use the -h flag. Optional arguments are indicated by brackets, and files are indicated by carrot symbols.

For example, the following command demonstrates the full suite of options for generating a numerical phantom in the image or k-space domain:

In [ ]:
! bart phantom -h

The `phantom` tool includes the option `-k` to create it in k-space, and `-x` to specify the size.

## Simulation with BART
###  BART Sequence

We can simulate a sequence with the *seq*-command. By default, a GRE (RF random spoiled) imaging block is computed.

If we do not modify the sequence parameters via the commandline-interface, a default sequence configuration is used.
Parameters can be found in *bart*: src/seq/config.c (or expand the following cell)


**Additional info (debug print)**:

Sequence blocks are composed from three types of events, namely RF pulses, ADCs, and gradients, all parametrized by start-, mid- and endtime. Gradients are further defined by an amplitude for all three directions. Both, RF and ADC-events, include phase and frequency information, enabling slice-shifted excitation and FOV-shifted acquisition. RF events are mapped to a pulse shape and ADCs are described by dwelltime, samples and position.




In [ ]:
! BART_DEBUG_LEVEL=5 bart seq -r 1 --TE 2e-3 --TR 3.5e-3 --FOV 0.256 --BR 256 adc grad moments

In [ ]:
plot_grad_from_cfl('grad', 3.5e-3)
plot_mom_from_cfl('moments', 3.5e-3)

### Non-Cartesian trajectory

For the acquisition of an image we acquire multiple radial spokes.
The *seq*-command provides us also with relevant information for the reconstruction, such as ADC timings/phase and the k-space trajectory.



In [ ]:
%%bash
bart seq -r377 --raga adc grad moments

bart show -m adc

bart extract 0 0 3 adc seq_traj

# bart traj -x 256 -o 2. -y 377 -r -D trj_ref
# bart nrmse trj_ref seq_traj_lin

In [ ]:
plot_grad_from_cfl('grad', 3.5e-3)
plot_mom_from_cfl('moments', 3.5e-3)

### Phantom simulation 

For the acquisition of an image we acquire multiple radial spokes.
The `seq`-command provides us also with relevant information for the reconstruction, such as ADC timings/phase and the k-space trajectory.

Using the `phantom`-tool and the `seq`-provided trajectory, we create an artificial k-space


https://github.com/mrirecon/bart-workshop/blob/master/ismrm2023/digital-objects/digital-objects.ipynb

In [ ]:
%%bash

# Simulation Parameters
#       Run `bart sim --seq h` for more details
TR=0.0035       # Repetition Time [s]
TE=0.0021       # Echo Time [s]
REP=600         # Number of repetitions

bart sim --ODE --seq FLASH,TR=$TR,TE=$TE,Nrep=$REP,FA=10 -1 0.1:2:11 -2 1:1:1 - | bart slice 5 $((REP-1)) - simu


Additional: explicitly defining the numerical phantom parameters.

In [ ]:
%%bash

# # Simulation Parameters
# #       Run `bart sim --seq h` for more details
# TR=0.0035       # Repetition Time [s]
# TE=0.0021       # Echo Time [s]
# REP=600         # Number of repetitions

# # Relaxation parameters for Brain phantom
# T1=(3 2.0 1.8 1.6 1.4 1.2 1.0 0.8 0.6 0.4 0.2)
# T2=(0.2  0.125  0.105  0.085  0.065 0.055 0.045 0.035 0.025 0.015 0.01)
# Run Simulations
# for i in `seq 0 10`; do

# 	bart sim --ODE --seq FLASH,TR=$TR,TE=$TE,Nrep=$REP,FA=8 -1 ${T1[$i]}:${T1[$i]}:1 -2 ${T2[$i]}:${T2[$i]}:1 - |\
# 	bart slice 5 $((REP-1)) - tmp_simu$(printf "%02d" $i)
# done
# bart join 6 $(ls tmp_simu*.cfl | sed -e 's/\.cfl//') simu

Combining simulated signal and basis geometries to a numerical k-space phantom can be achieved with the `phantom` tool in combination with the `-b` flag.

In [ ]:
%%bash

bart phantom -k -T -b -t seq_traj basis_geom
bart fmac -s 64 basis_geom simu ksp


And perform a simple reconstruction (one coil, inverse NUFFT)

In [ ]:
! bart nufft -i -t seq_traj ksp img
show_img(cfl.readcfl('img'), title='Reconstructed image')

### FOV shifts

With the ADC timing and the phase information,
we can also simulate (and validate) FoV shifts of our sequence by checking the phase of each ADC sample.

(For this purpose, we have to turn off RF spoiling.)

In [ ]:
! bart seq -r377 -S 0.05:-0.12:0 --raga --no-spoiling adc grad moments
! bart slice 0 4 adc adc_phase
! bart fmac adc_phase ksp ksp_offcenter

# ! bart fovshift -s 0.05:-0.12:0 -t seq_traj ksp ksp_offcenter

! bart nufft -i -t seq_traj ksp_offcenter img_offcenter

! bart join 1 img img_offcenter img_off

show_img(cfl.readcfl('img_off'), title='Original vs. offcenter image')


### Affine registration

Now, we try to estimate the rigid-body motion between these two images and correct for the shifts and check if we can recover the ground truth

In [ ]:
! bart affinereg  img img_offcenter affine_matrix
! bart slice 1 3 affine_matrix shift
! bart fovshift -S shift -t seq_traj ksp_offcenter ksp_corr

In [ ]:
! bart nufft -i -t seq_traj ksp_corr img_corr


In [ ]:
! bart saxpy -- -1 img img_offcenter	img_diff1
! bart saxpy -- -1 img img_corr 		img_diff2

! bart join 1 img img_offcenter  img_diff1 img1
! bart join 1 img img_corr 	img_diff2 img2

! bart join 0 img1 img2 img

show_img(cfl.readcfl('img'), title='Original vs. offcenter (uncorrected/corrected) vs. difference')

## Reconstruction

A complete tutorial for creating and reconstructing phantom data can be found in the [1st BART Webinar Materials](https://github.com/mrirecon/bart-webinars/blob/master/webinar1/day1_advanced_recon.ipynb)

We start with a simple example that already demonstrates some useful functions in BART:

- Creation of synthetic k-space data using the `phantom` tool
- Calibration of sensitivities using NLINV using the `ncalib` tool
- Parallel imaging reconstruction using the `pics` tool

On the first step, we will create synthetic multi-coil k-space data using the `phantom` tool. We use the '-k' option to specify that we want to simulate k-space data, -s to set the number of coils. Further dimensions, such as readout and phase-encoding, are specified by the trajectory.

In [ ]:
# create fully-sampled non-Cartesian multi-coil phantom in k-space
! bart seq -r377 -t 100 --chrono --raga adc_us grad moments
! bart extract 0 0 3 adc_us - | bart extract 2 0 100 -  seq_traj_us

! bart phantom -k -s8 -T -b -t seq_traj_us basis_geom2
! bart fmac -s 64 basis_geom2 simu ksp_us

! bart reshape $(bart bitmask 2 3) $((8 * 100)) 1 ksp_us - | bart transpose 0 1 - - | bart transpose 1 2 - ksp_usR
show_img(cfl.readcfl('ksp_usR'), title='multi-channel k-space phantom', vmax=50, fsize=15)

Note that we used numpy broadcasting to automatically apply the mask to each coil. There is no undersampling mask along the readout direction.

After creating the under-sample data, we now first perform a simple reconstruction using a non-uniform inverse Fourier transform (`nufft`) and root-sum-of-squares coil combination (`rss`).

In [ ]:
# simple nuFFT + RSS reconstruction

# perform inverse nuFFT.
! bart nufft -i seq_traj_us ksp_us img_us

# RSS-combine
! bart rss $(bart bitmask 3) img_us img_rss

show_img(cfl.readcfl('img_rss'), title='RSS Recon')

The result is (of course) not very good. The image is corrupted with streaking artifacts due to the radial undersampling.

Let's try a more advanced reconstruction technique. We will first determine the coil sensitivities. Here, we use the NLINV $^1$ algorithm to obtain the sensitivity maps for each coil. We will now have a closer look at the NLINV `ncalib` tool.


$^1$ Uecker M, Hohage T Block KT, Frahm J. Image reconstruction by regularized nonlinear inversion-joint estimation of coil sensitivities and image content. Magn Reson Med. 2008;60(3):674-82.

In [ ]:
! bart ncalib -h

We normalize coil sensitivities in this example by using the option `-N`.

In [ ]:
# compute sensitivity maps with NLINV
! bart ncalib -N -t seq_traj_us ksp_us sens_maps

! bart reshape $(bart bitmask 1 3) $((8 * 256)) 1 sens_maps sens_mapsR
show_img(cfl.readcfl('sens_mapsR'), title='Sensitivity maps', fsize=15)

Given the sensitivities, we can use the `pics` command to perform iterative parallel imaging reconstruction using various regularization terms. It solves the folllowing minimization problem:

$\arg\min_x ||PFS x - y|| + \alpha R(x)$, where:
- $P$ is the sampling operator
- $F$ is the Fourier transform
- $S$ is the multiplication with the sensitivity maps
- $x$ is the unknown image
- $y$ are the acquired multi-coil k-space data
- $R$ is a regularization functional
- $\alpha$ is the regularization weighting parameter

A popular regularization is $\ell_1$ wavelet , i.e. $R(x) = ||Wx||_1$.

The objective function then corresponds to a SENSE reconstruction.

In [ ]:
# parallel imaging reconstruction using l1 regularization; -e Scale stepsize based on max. eigenvalue; 
! bart pics -S -e -l1 -r 0.001 -i 50 -t seq_traj_us ksp_us sens_maps img_l1

show_img(cfl.readcfl('img_l1'), 'CG-SENSE reconstruction')

In [ ]:
# combine images for display
! bart join 1 img_rss img_l1 img2

show_img(cfl.readcfl('img2'), title='RSS vs CG-SENSE reconstructions', fsize=15)

## Data acquisition with BART / pulseq

The sequence can also be saved in pulseq format, including metadata such as ADC labels and triggering information.
Data can then be acquired with vendor-specific pulseq interpreters.

Let us check the *.seq* file

<!-- Inconsistencies in offcenter measurements can be corrected retrospectively by adding the measured FOV-shift to the BART command and adding this phase information to the data in a preprocessing step. -->

In [ ]:
! bart seq --TE 2e-3 --TR 3.5e-3 -r3 --FOV 0.256 --BR 256 adc grad moments three_spokes.seq
! head -n30 three_spoke.seq

In [ ]:
import pypulseq as pp

seq = pp.Sequence()
seq.read('three_spokes.seq')
seq.plot()

For a meaningful acquisition, we have to adjust the number of radial spokes and frames.

In [ ]:
! bart seq --TE 2e-3 --TR 3.5e-3 -r377 -f 3 --FOV 0.256 --BR 256 adc grad moments bart_raga377_frames3.seq


## Reconstructing data from the scanner

Because the BART file format is simple, it is possible to read data from many different sources. The BART command includes the `twixread` tool for basic reading of Siemens `.dat` files. We also include GE Orchestra libraries for reading P-File and ScanArchives, called [ox-bart](https://github.com/mrirecon/ox-bart).

In the data folder, we have already converted a k-space dataset from a vendor-specific format to BART format.

Here, `ksp` is a single full RAGA frame. `noise` is a single measurement line without excitation, eg. a noise adjustment scan.

By using `-r1` we read data in a non-Cartesian manner, eg. a singleton dimension (dim 0), readout in dimension 1, and spokes in dimension 2.


In [ ]:
%%bash

# bart twixread -A -r1 pulseq.dat ksp_raw; bart slice 10 1 ksp_raw ksp

echo "ksp:"
# use the show command to display dimensions
bart show -m data/ksp

echo "noise:"
# use the show command to display dimensions
bart show -m data/noise




In [ ]:
! bart reshape $(bart bitmask 1 3) $((16 * 512)) 1 data/ksp - | bart transpose 0 2 - kspR # only for visualization
show_img(cfl.readcfl('kspR'), title='k-space', vmax=.00001, fsize=40)

Based on the array sizes and dimensions, we can infer that the readout dimension is 512 samples, 377 acquired lines, and there are 16 coils.

#### Noise prewhitening & Coil Compression

Before reconstruction, it is a good idea to do noise pre-whitening. Most vendors should provide you with a noise pre-scan. If not, then you should complain to your vendor!

We can use the BART `whiten` tool to create a noise prewhitening matrix or apply this matrix already on the k-space.

To reduce the size of our dataset and therefore also decrease the computational complexity, we perform a coil compression with the `cc` command.

We will do coil compression in 2 steps. We will first use `cc` on the reference data to create the coil compression matrix. We will then use ``ccapply` to apply it to both the reference data and to the k-space data. This way we use the same coil compression matrix for both arrays.

By passing `-A` we choose to use all possible data and to reduce the dataset to 8 virtual coils with `-p`.


In [ ]:
! bart cc -h

In [ ]:
%%bash

bart whiten data/ksp data/noise ksp_white

bart cc -A -M ksp_white cc_mat
bart ccapply -p 8 ksp_white cc_mat ksp_cc_white

bart show -m ksp_cc_white


In [ ]:
! bart reshape $(bart bitmask 1 3) $((8 * 512)) 1 ksp_cc_white - | bart transpose 0 2 - kspR # only for visualization
show_img(cfl.readcfl('kspR'), title='k-space', vmax=2, fsize=40)

#### Trajectory

As shown above, the commandline for generation of the sequence is stored in the *pulseq*-file.
So by running the command, we can regenerate the trajectory.

For our in-house vendor sequence, where BART is integrated as a dynamic linked library, we recover UI/system parameters from the raw-data and generate a *seq* command.


In [ ]:
! head pulseq.seq

In [ ]:
%%bash
bart seq --IR_NON --init_delay=10 --TR 3.4e-3 --TE 2.04e-3 --raga --tiny 2 -r377 -t1800 --FOV .220 --dwell 4.6e-6 adc gradients moments irraga_FOV220.seq 
bart show -m adc

bart slice 10 1 adc adc_frame # analogous to ksp

bart extract 0 0 3 adc_frame seq_traj

#### Check with NUFFT

In [ ]:
! bart nufft -i seq_traj ksp_cc_white img_nufft

! bart reshape $(bart bitmask 1 3) $((8 * 256)) 1 img_nufft imgR # only for visualization
show_img(cfl.readcfl('imgR'), title='nuFFT reconstruction', vmax=2, fsize=40)

#### FOV correction

In [ ]:
! bart seq -S 0.04:0:0 --no-spoiling -r377 adc
! bart slice 0 4 adc - | bart fmac ksp_cc_white - ksp_corr

! bart nufft -i seq_traj ksp_corr img_nufft

! bart reshape $(bart bitmask 1 3) $((8 * 256)) 1 img_nufft imgR # only for visualization
show_img(cfl.readcfl('imgR'), title='nuFFT reconstruction (FOV corrected)', vmax=4, fsize=40)

#### Gradient delay correction


In [ ]:
! bart estdelay -R seq_traj ksp_corr gd
! bart trajcor -V gd -O seq_traj seq_traj_gd

#### Coil sensitivity estimation


In [ ]:
# estimate coil sensitivities
! bart ncalib -N -t seq_traj_gd ksp_corr sens_maps

In [ ]:
# display the coils
! bart reshape $(bart bitmask 1 3) $((8 * 256)) 1 sens_maps sens_mapsR
show_img(cfl.readcfl('sens_mapsR'), title='Coil-compressed sensitivity maps', fsize=15)

#### Parallel imaging reconstruction
We again use `pics` with l2 regularization. This time we pass an approximate scale factor directly to the `pics` command.

In [ ]:
! bart pics -d4 -SeH -l1 -i50 -r 18e-4 -t seq_traj_gd ksp_corr sens_maps reco

#### Visualize reconstruction

In [ ]:
show_img(cfl.readcfl('reco'), title='l1-wavelet Reconstruction', vmax=7)

## RECO2'

In [ ]:
%%bash
bart seq --FOV 0.256 --BR 256 -r 377 -f 1 --dwell 10e-6 --FA 50 --raga --spoiled --TR 250e-3 --TE 3e-3 samples

In [ ]:
%%bash
bart vec 1 1 0 a
bart extract 0 0 3 samples - | bart fmac - a seq_traj


bart whiten data/ksp2 data/noise2 ksp_white

bart cc -A -M ksp_white cc_mat
bart ccapply -p 8 ksp_white cc_mat ksp_cc_white

bart show -m ksp_cc_white

In [ ]:
! bart reshape $(bart bitmask 1 3) $((8 * 512)) 1 ksp_cc_white - | bart transpose 0 2 - kspR # only for visualization
show_img(cfl.readcfl('kspR'), title='k-space', vmax=2, fsize=40)

In [ ]:
! bart estdelay -R seq_traj ksp_cc_white gd
! bart show gd
! bart trajcor -V gd -O seq_traj seq_traj_gd


In [ ]:
! bart ncalib -N -t seq_traj_gd ksp_cc_white sens_maps


In [ ]:
# display the coils
! bart transpose 0 1 sens_maps - | bart flip 1 - -  | bart reshape $(bart bitmask 1 3) $((8 * 256)) 1 - sens_mapsR
show_img(cfl.readcfl('sens_mapsR'), title='Coil-compressed sensitivity maps', fsize=15)

In [ ]:
! bart pics -d4 -SeH -l1 -i50 -r 1e-4 -t seq_traj_gd ksp_cc_white sens_maps reco

In [ ]:
! bart transpose 0 1 reco - | bart flip 1 - reco2
show_img(cfl.readcfl('reco2'), title='l1-wavelet Reconstruction', vmax=15)